In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import pickle

In [3]:
print(tf.__version__)
print(keras.__version__)

2.0.0-alpha0
2.2.4-tf


In [27]:
with open("baudelaire.txt") as f:
    baudelaire_text = f.read()

In [28]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([baudelaire_text])

In [29]:
tokenizer.texts_to_sequences(["Bonjour"])

[[22, 11, 5, 27, 11, 6, 8]]

In [47]:
tokenizer.sequences_to_texts([[22, 11, 5, 27, 11, 6, 8]])

['b o n j o u r']

In [31]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [32]:
max_id

70

In [33]:
dataset_size

1

In [34]:
[encoded] = np.array(tokenizer.texts_to_sequences([baudelaire_text])) - 1

In [35]:
encoded

array([30,  0, 14, ...,  0, 28, 13])

In [36]:
len(encoded)

235437

In [49]:
train_size = len(encoded) * 90 // 100

In [50]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [51]:
dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [52]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [53]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [54]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [55]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [56]:
dataset = dataset.prefetch(1)

In [57]:
dataset

<PrefetchDataset shapes: ((None, None, 70), (None, None)), types: (tf.float32, tf.int64)>

In [58]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
   1293/Unknown - 464s 359ms/step - loss: 2.1703

KeyboardInterrupt: 

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["Bonjour comment all"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
        return text

In [ ]:
print(complete_text("t", temperature=0.2))

In [ ]:
print(complete_text("l", temperature=1))

In [ ]:
print(complete_text("l", temperature=2))

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=True, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])